# Using tool to get information about a country, from wikipedia




## Import libaries and prepare the API credential

In [1]:
import tempfile
import shutil
import os
import subprocess
import requests

from autogen import ConversableAgent, register_function, GroupChatManager, GroupChat
from autogen.coding import LocalCommandLineCodeExecutor

with open('openai.credential', 'r') as file:
    key = file.read()
MODEL = 'gpt-4o'

## function to call Wiki API


In [2]:
def describe_country(country_name:str)->str:
    url = "https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro&explaintext&redirects=1&titles="+country_name
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        json_data = response.json()  # Extract JSON content
        page = next(iter(json_data["query"]["pages"].values()))
        return page["extract"]
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Other error occurred: {err}")


## Agent 1: Get country description from wikipedia

In [3]:

wiki_tool = ConversableAgent(
    "wiki_tool",
    system_message = "You are a helpful assistant. You may decide to use some external tools",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config=False
)

wiki_tool_driver = ConversableAgent(
    "wiki_tool_driver",
    system_message = "your only task is to call a tool for wiki service to obtain the description of one country",
    llm_config = {"config_list": [{"model": MODEL, "api_key": key}]},
    code_execution_config=False
)

register_function(
    describe_country,
    caller = wiki_tool_driver,
    executor = wiki_tool,
    description = "get country description from wiki"
)


def second_last_msg(sender: ConversableAgent, recipient: ConversableAgent, summary_args: dict):
    return sender.chat_messages[recipient][-2]["content"]


nested_chats = [
    {
        "recipient": wiki_tool_driver,
        "max_turns": 2,
        "summary_method": second_last_msg
    }
]

wiki_tool.register_nested_chats(
    nested_chats, 
    trigger = lambda sender: sender not in [wiki_tool_driver]
)

## Agent 2: Extract countries mentioned in the country description

In [4]:
relevant_country_extractor = ConversableAgent(
    "relevant_country_extractor",
    system_message = "You are helpful assist. When you receive a text about a country, try to find out what countries"
    "are mentioned, and list them one by one, each in a line, followed by 'Here is a list of countries'",
    llm_config = {"config_list": [{"model": MODEL, "api_key": key}]},
    code_execution_config=False
)

## Agent 3: Code executor that executes python code

In [5]:
temp_dir = tempfile.TemporaryDirectory()
print(temp_dir.name)

subprocess.run(["cp", "population.csv", temp_dir.name+"/"])

os.listdir(temp_dir.name)

executor = LocalCommandLineCodeExecutor(
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

# Create an agent with code executor configuration.
code_executor = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
    is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
)

/tmp/tmp9dps21yu


## Agent 4: The data scientist agent that knows how to write python code

In [6]:
data_scientist_system_message = """You are a data scientist, and you can solve problems by writing python code. 
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply 'TERMINATE' in the end when the anser is verified.
"""

data_scientist = ConversableAgent(
    "data_scientist_agent",
    system_message = data_scientist_system_message,
    llm_config = {"config_list": [{"model": MODEL, "api_key": key}]},
    code_execution_config=False,  # Turn off code execution for this agent.
    is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
    
)

## Agent 5: A story teller that reads data and generate a story

In [7]:
story_teller = ConversableAgent(
    "story_teller",
    system_message = "Your are a analyzer. If you see a set of data, you try to tell a story about what you understand about the data",
    llm_config = {"config_list": [{"model": MODEL, "api_key": key}]},
    code_execution_config=False
)

## Agent 6: the "architect", which can break down a large task into small tasks for different agents

In [8]:
architect = ConversableAgent(
    "architect",
    system_message = "You are an AI-based architect that make plans for the whole group."
    "When you get a task, break down it into sub tasks, and each can be performed by one of your 'partner agents'."
    "You will get an introduction about what each of your partner agents can do.",
    llm_config = {"config_list": [{"model": MODEL, "api_key": key}]},
    code_execution_config=False,  # Turn off code execution for this agent.
    human_input_mode = "ALWAYS"
)


## Now we build the group
We give each agent a short description, which they will use later to introduce themselves.

In [9]:
wiki_tool.description = "I can describe some key information about a country, as provided by Wikipedia"
relevant_country_extractor.description = "I can extract what other countries are mentioned in a description"
code_executor.description = "I can execute python code"
data_scientist.description = "I can write python code to solve problems, such as reading data from a csv file"
architect.description = "I am the architect, and I break down a task into subtasks. I should always be the one to speak first, to make plan for sub-tasks"
story_teller.description = "I can tell a story from the data I see"


group_chat = GroupChat(
    agents = [wiki_tool, relevant_country_extractor, code_executor, data_scientist, story_teller, architect],
    messages = [],
    max_round = 20,
    send_introductions = True
)

group_chat_manager = GroupChatManager(
    groupchat = group_chat,
    llm_config = {"config_list": [{"model": MODEL, "api_key": key}]}
)


## Here we give the task.
We need someone in the group to forward the task to the group manager

In [10]:
chat_result = relevant_country_extractor.initiate_chat(
    group_chat_manager,
    max_round = 6,
    message = "Here is the high-level task: I want to compare number of newborns of Germany and some of its relevant countries"
    "(i.e., those that are mentioned with Germany in Wiki), in every decade from 1950 to 2020."
    "Some population data can be obtained from population.csv. You may need to check the csv structure first. "
    "Regarding the output, I would like to see the data on the console, have a trend.png file with a chart to visualize the data"
    "and then a trend.pdf file which contains the previous chart, together with a story that explains it"
)

relevant_country_extractor (to chat_manager):

Here is the high-level task: I want to compare number of newborns of Germany and some of its relevant countries(i.e., those that are mentioned with Germany in Wiki), in every decade from 1950 to 2020.Some population data can be obtained from population.csv. You may need to check the csv structure first. Regarding the output, I would like to see the data on the console, have a trend.png file with a chart to visualize the dataand then a trend.pdf file which contains the previous chart, together with a story that explains it

--------------------------------------------------------------------------------

Next speaker: architect


>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
architect (to chat_manager):

Alright, let's break down this task into specific subtasks and assign them to the appropriate partner agents in the team.

1. **Extract relevant countries:** Identify the countries that are mentioned with Germany in the Wi

In [11]:
subprocess.run(["cp", temp_dir.name+"/trend.pdf", "./"])

CompletedProcess(args=['cp', '/tmp/tmp9dps21yu/trend.pdf', './'], returncode=0)